In [1]:
%run "000_includez.ipynb"

In [2]:
from skimage.io import imread as skread 
import skimage
import skimage.filters as skfilters

In [3]:
class SrcImage:
#     ## image to operate on
    def __init__(self, src_path):
#         self.img = skread( src_path ) 
        ## using eye retina image from skimage pkg
        self.img_full = skimage.data.retina()
        self.img = self.img_full[300:700, 700:900] 
        
    @property 
    def gray_float_array(self): 
        return skimage.img_as_float32( skimage.color.rgb2gray( self.img ) ) * -1
    
    def denoised(self, sig=1, gray=True):
        # Gaus will smooth but at expense of edges so keep sig small
        ## TOD: skimage Vs ndimage
        if gray:
            return skfilters.gaussian(self.gray_float_array, sig) 
        else:
            return skfilters.gaussian(self.img, sig)
    
    def noisy(self, p=0.04):
        outi = self.img.copy()
        outi = outi + p* outi.std() * np.random.random(outi.shape)
        return outi
    
    @property
    def stats(self): 
        return f"---Image Stats---\n Shape: {self.img.shape} \n Type: { type( self.img )} \n Mean: { np.mean( self.img )} \n Median: {np.median( self.img )} \n Max: {np.max( self.img )} \n Min: {np.min( self.img )} "
    
    def show(self, cmap='gray'): 
        if (cmap is None):
            plt.imshow(self.img)
        else:
            plt.imshow(self.gray_float_array, cmap=cmap) 
        plt.axis('off')
        
    def show_with_hist(self, cmap='gray', binz=None):
        plt.subplot(1,2,1)
        self.show(cmap)
        
        plt.subplot(1,2,2)
        img = self.gray_float_array
        plt.hist(img.flatten()*(1/img.max()), bins=binz)
        plt.title(f"Histogram: {binz} bins")
        plt.tick_params(axis='y', which='both', labelleft=False, labelright=False)
        plt.subplots_adjust(wspace=0, hspace=0)
        

In [4]:
class AFilter:
    ## apply filters and report on performance
    def __init__(self, filter_fxn, filter_argz, filter_name='', mode=-1):
        self.filter_fxn = filter_fxn
        self.filter_argz = filter_argz
        self.filter_name = filter_name 
        self.mode = mode 
        
    def apply(self, img):
        self.src_img  = img
        self.result = self.filter_fxn(img, **self.filter_argz) * self.mode 
        return self.result 
    
    def compare_diff(self, img_list, img_titlez=None, show=True):
        outsiez = []
        def diff(inimg):
            return np.abs( self.result - img ) * self.mode
        
        for img in img_list:
            outsiez.append( diff( img ) ) 
            
        if show:
            show_image_list( outsiez, titlez=img_titlez, nc = len(outsiez))
        
        return outsiez ##a list of images 
    
    def compare_stats(self, img_list, diff_fx=None,  img_titlez=None, show=True):
        outsiez = []
        def diff(inimg): ## MSE
            return np.round( (np.square( self.result - img )).mean(), 4)
        
        def print_stats(stats, labels):
            fx = "MSE" if diff_fx is None else type(diff_fx)
            rep = [ ]
            for i, stat in enumerate(stats):
                s = f"img {i+1}" if labels is None else labels[i]
                rep.append(f"{fx}: ({self.filter_name} - {s}) \t{stat}")
            return " \n".join(rep)
        
        for img in img_list:
            if diff_fx is None:
                outsiez.append( diff( img ) ) 
            else:
                outsiez.append( diff_fx( img ) ) 
  
        reportez = print_stats( outsiez, img_titlez)
        if show:
            print( reportez ) 
        
        return reportez ## a string
    
    def product_mash(self, img_list, img_titlez=None, show=True):
        t = "" if img_titlez is None else ", ".join(img_titlez)
        
        outsiez = self.result.copy()
        for img in img_list:
            outsiez = (outsiez * img)  * self.mode
        
        if show:
            show_image_list( [self.result, outsiez], titlez=[f"{self.filter_name}", f"product_mashup - {t}"], nc = 2)
        
        return outsiez ## and image array 
    
    def sum_mash(self, img_list, img_titlez=None, show=True):
        t = "" if img_titlez is None else ", ".join(img_titlez)
        
        outsiez = self.result.copy()
        for img in img_list:
            outsiez = (outsiez + img ) #* self.mode
            
        if show:
            show_image_list( [self.result, outsiez], titlez=[f"{self.filter_name}", f"sum_mashup - {t}"], nc = 2)
        
        return outsiez ##an image array 